examples of different ways how to select signals for CAN log extraction

In [6]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import canmatrix.formats
from etils import ecolab
from pathlib import Path
from collections import defaultdict
import os
from utils.dbc_db import DbcDb
import  utils.can_log_utils as utls
from asammdf import MDF

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#  set inputs
dbc_dir=r'dbc\ucs_dbc-0.12.0.0'
report_file=r'output\signal_selection\VehicleReadyCheck\df_msg.xlsx'
map0_sheet_name='ALL_DBC_CAN_MAP'
log_file1=r'input\canlog_38837_20241003_113923_move.mf4'
dbc_db=DbcDb(dbc_dir)

total number of id: 1715
total number of dbc.msg: 3143
number of dublicated id: 1245
number of dublicated pgn: 1210
dbc db loaded


In [8]:
# prepare can maps
can_map0,can_map_ch =utls.get_can_map(report_file,map0_sheet_name)
can_map0.shape;
can_map0.head();

(6, 3)

,dbc,CAN1,CAN2_VEH
0,DTS_Status1.dbc,False,True
1,DTS_Status2.dbc,True,False
2,DTS_Status3.dbc,True,False
3,GS.dbc,False,True
4,SimpVSM.dbc,False,True


In [9]:
# get messages only for selected dbc
cols2join=['dbc','msg']
df_dbc_msg=dbc_db.df[cols2join].copy()

df_dbc_msg=pd.merge(can_map0,df_dbc_msg,on='dbc')
df_dbc_msg=df_dbc_msg.reindex(columns=['dbc','msg']+can_map_ch)
df_dbc_msg.shape;
df_dbc_msg.head();
with pd.ExcelWriter(report_file,mode='a',if_sheet_exists='replace') as writer:  
    df_dbc_msg.to_excel(writer,sheet_name='dbc_msg_auto',index=False)
print('need to select which messages need to be taken, make it in sheet dbc_msg_sel')

(10, 10)

,dbc,msg,CAN1,CAN2_VEH,CAN3,CAN4,CAN5,CAN6,CAN7_BRK,CAN8
0,DTS_Status1.dbc,DTS_Status1,False,True,NaN,NaN,NaN,NaN,NaN,NaN
1,DTS_Status2.dbc,DTS_Status2,True,False,NaN,NaN,NaN,NaN,NaN,NaN
2,DTS_Status3.dbc,DTS_Status3,True,False,NaN,NaN,NaN,NaN,NaN,NaN
3,GS.dbc,VECTOR__INDEPENDENT_SIG_MSG,False,True,NaN,NaN,NaN,NaN,NaN,NaN
4,GS.dbc,GS_State,False,True,NaN,NaN,NaN,NaN,NaN,NaN


need to select which messages need to be taken, make it in sheet dbc_msg_sel


In [10]:
# get selected dbc_msg  as new input to select specific messages from specific CAN channels
df_dbc_msg,can_map_ch=utls.get_can_map(report_file,'dbc_msg_sel',2)
can_map_ch;
df_dbc_msg.shape;
df_dbc_msg.head();

['CAN1', 'CAN2_VEH']

(7, 4)

,dbc,msg,CAN1,CAN2_VEH
0,DTS_Status1.dbc,DTS_Status1,False,True
1,DTS_Status2.dbc,DTS_Status2,True,False
2,DTS_Status3.dbc,DTS_Status3,True,False
3,GS.dbc,GS_State,False,True
4,SimpVSM.dbc,VSMState,False,True


In [11]:
# next create table with signals
cols2join=['dbc','msg','signal']
df_msg_sig=dbc_db.df_sig[cols2join].copy()
df_msg_sig=pd.merge(df_dbc_msg,df_msg_sig,on=['dbc','msg'],how='left')
df_msg_sig=df_msg_sig[cols2join]
df_msg_sig['IsSelected']=1
df_msg_sig.shape;
df_msg_sig.index;
df_msg_sig.columns;
df_msg_sig.head();
with pd.ExcelWriter(report_file,mode='a',if_sheet_exists='replace') as writer:  
    df_msg_sig.to_excel(writer,sheet_name='dbc_msg_sig_auto',index=False)


(111, 4)

RangeIndex(start=0, stop=111, step=1)

Index(['dbc', 'msg', 'signal', 'IsSelected'], dtype='object')

,dbc,msg,signal,IsSelected
0,DTS_Status1.dbc,DTS_Status1,DTS_Standstill,1
1,DTS_Status1.dbc,DTS_Status1,DT_ReadyToSleep,1
2,DTS_Status1.dbc,DTS_Status1,DTS_SystemStatus,1
3,DTS_Status1.dbc,DTS_Status1,DTS_AccrPedlSw,1
4,DTS_Status1.dbc,DTS_Status1,DTS_BrkPedlPosn,1


In [12]:
df_msg_sig,can_map_ch=utls.get_can_map(report_file,'dbc_msg_sig_sel',3)
df_msg_sig.shape;
df_msg_sig.head()


(102, 4)

,dbc,msg,signal,IsSelected
0,DTS_Status1.dbc,DTS_Status1,DTS_Standstill,True
1,DTS_Status1.dbc,DTS_Status1,DT_ReadyToSleep,True
2,DTS_Status1.dbc,DTS_Status1,DTS_SystemStatus,True
3,DTS_Status1.dbc,DTS_Status1,DTS_AccrPedlSw,True
4,DTS_Status1.dbc,DTS_Status1,DTS_BrkPedlPosn,True


now try to extract data from log file

In [13]:
mdf1 = MDF(log_file1)

In [14]:
# prepare dbc maps
can2ch_maps={'CAN1':1,'CAN2_VEH':2}
(_,can_db1)=utls.get_can_map2extract(dbc_dir,can2ch_maps,report_file,sheet=map0_sheet_name)
can_db1;

{'CAN': [(WindowsPath('dbc/ucs_dbc-0.12.0.0/DTS_Status2.dbc'), 1),
  (WindowsPath('dbc/ucs_dbc-0.12.0.0/DTS_Status3.dbc'), 1),
  (WindowsPath('dbc/ucs_dbc-0.12.0.0/DTS_Status1.dbc'), 2),
  (WindowsPath('dbc/ucs_dbc-0.12.0.0/GS.dbc'), 2),
  (WindowsPath('dbc/ucs_dbc-0.12.0.0/SimpVSM.dbc'), 2),
  (WindowsPath('dbc/ucs_dbc-0.12.0.0/SimpVSM_DBG.dbc'), 2)]}

In [15]:
mdf1_extracted = mdf1.extract_bus_logging(database_files=can_db1)

In [16]:
mdf1_all_signames=[list(can_signal.display_names.keys())[0] for can_signal in mdf1_extracted];

['CAN1.DTS_Status2.DTS_Status2_counter',
 'CAN1.DTS_Status2.DTS_Status2_checksum',
 'CAN1.DTS_Status2.DTS_Stalk_GS_Input',
 'CAN1.DTS_Status2.DTS_GS_Human_RND_Input',
 'CAN1.DTS_Status2.DTS_IDU_CommStatus',
 'CAN1.DTS_Status2.DTS_ParkBrakeSt',
 'CAN1.DTS_Status2.DTS_ParkBrakeCmd',
 'CAN1.DTS_Status2.DTS_IDU_InterlockSt',
 'CAN1.DTS_Status2.DTS_IDU_InterlockCmd',
 'CAN1.DTS_Status2.DTS_IDU_DirectionSt',
 'CAN1.DTS_Status2.DTS_IDU_DirectionCmd',
 'CAN1.DTS_Status2.DTS_IDU_HV_CheckState',
 'CAN1.DTS_Status2.DTS_SysSt_ParkLock',
 'CAN1.DTS_Status2.DTS_SysSt_IDU',
 'CAN1.DTS_Status3.DTS_Status3_counter',
 'CAN1.DTS_Status3.DTS_Status3_checksum',
 'CAN1.DTS_Status3.DTS_IDU_DCCurrent',
 'CAN1.DTS_Status3.DTS_IDU_MotTemp1',
 'CAN1.DTS_Status3.DTS_IDU_InvTemp1',
 'CAN1.DTS_Status3.DTS_IDU_DCVoltage',
 'CAN2.DTS_Status1.DT_ReadyToSleep',
 'CAN2.DTS_Status1.DTS_SystemStatus',
 'CAN2.DTS_Status1.DTS_AccrPedlSw',
 'CAN2.DTS_Status1.DTS_BrkPedlPosn',
 'CAN2.DTS_Status1.DTS_IgnitionStatus',
 'CAN2.DT

In [ ]:
import re
mdf1_msgs=defaultdict(lambda: defaultdict(list))
mdf1_msgs_grp=defaultdict(lambda: defaultdict(int))
mdf1_can_sa_msgs_grp=defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
mdf1_can_sa_msgs=defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
mdf1_can_sa_msgs_sigs_idx=defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(tuple))))
#                               can             sa                  msg                 sig

mdf1_all_signames=[list(can_signal.display_names.keys())[0] for can_signal in mdf1_extracted]

for can_signal in mdf1_extracted:
    can_msg_sig=list(can_signal.display_names.keys())[0]
    msg=can_msg_sig.split('.')
    m_can=msg[0]
    m_msg=msg[1]
    m_sig=msg[2]
    src=can_signal.source.name
    src_sa='NO'
    m_gr_ch_idx=(can_signal.group_index, can_signal.channel_index)
    if 'SourceAddress' in src:
        # get SA address of the sender
        if '0x' in src:
            m=src.split('0x')
            if len(m)>1:
                src_sa=m[1]
            else:
                src_sa=src
        else:
                src_sa=src
        m=mdf1_can_sa_msgs

    else:
        src_sa='0x0'
    if msg[2] not in  mdf1_can_sa_msgs[msg[0]][src_sa][msg[1]]:
        mdf1_can_sa_msgs[msg[0]][src_sa][msg[1]].append(msg[2])
        
    if m_sig not in  mdf1_can_sa_msgs_sigs_idx[m_can][src_sa][m_msg]:
        mdf1_can_sa_msgs_sigs_idx[m_can][src_sa][m_msg][m_sig]=m_gr_ch_idx
        
    if msg[2] not in mdf1_msgs[msg[0]][msg[1]]:
        mdf1_msgs[msg[0]][msg[1]].append(msg[2])
    # detect multiple groups
    grp_sig_id=mdf1_extracted.channels_db[can_msg_sig]
    grp_sig_id_sum=sum(len(x) for x in grp_sig_id)
    if grp_sig_id_sum>2:
        print(f"group already exist for {can_signal.display_names} {grp_sig_id}. Take the first one")

    mdf1_msgs_grp[msg[0]][msg[1]]=grp_sig_id[0][0]
    mdf1_can_sa_msgs_grp[msg[0]][src_sa][msg[1]]=grp_sig_id[0][0]

mdf1_msgs_grp;
mdf1_msgs;

defaultdict(<function __main__.<lambda>()>,
            {'CAN1': defaultdict(int, {'DTS_Status2': 0, 'DTS_Status3': 1}),
             'CAN2': defaultdict(int,
                         {'DTS_Status1': 2,
                          'GS_State': 3,
                          'VSMState': 4,
                          'VSM_DEBUG_OUT_1': 5})})

defaultdict(<function __main__.<lambda>()>,
            {'CAN1': defaultdict(list,
                         {'DTS_Status2': ['DTS_Status2_counter',
                           'DTS_Status2_checksum',
                           'DTS_Stalk_GS_Input',
                           'DTS_GS_Human_RND_Input',
                           'DTS_IDU_CommStatus',
                           'DTS_ParkBrakeSt',
                           'DTS_ParkBrakeCmd',
                           'DTS_IDU_InterlockSt',
                           'DTS_IDU_InterlockCmd',
                           'DTS_IDU_DirectionSt',
                           'DTS_IDU_DirectionCmd',
                           'DTS_IDU_HV_CheckState',
                           'DTS_SysSt_ParkLock',
                           'DTS_SysSt_IDU'],
                          'DTS_Status3': ['DTS_Status3_counter',
                           'DTS_Status3_checksum',
                           'DTS_IDU_DCCurrent',
                           'DTS_IDU_MotTemp1'

In [ ]:
def decode_bytes(df):
    for col in df.columns:
        if df[col].dtype == object and df[col].dropna().apply(lambda x: isinstance(x, bytes)).all():
            df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
    return df

In [ ]:
msgs1_df=defaultdict(lambda: defaultdict(list))


for can,can_msgs in mdf1_msgs_grp.items():
    # print(can)
    
    for msg, grp in can_msgs.items():
        df=mdf1_extracted.get_group(grp,use_display_names=False)
        df=decode_bytes(df)
        # dT=np.diff(df.index,prepend=0)
        # if len(dT)>1:
            # dT[0]=dT[1]
        
        # df['dT']=dT
        # df.to_csv(f'output/export/{can}.{msg}.csv')
        # msgs1_df[can][msg]=df
        msgs1_df[msg]=df
        # sigs2sel=[".".join([can,msg,sig]) for sig in sigs]
        # sigs_selected=mdf1_extracted.filter(sigs2sel)
        
df_DTS_Status1=mdf1_extracted.get_group(mdf1_msgs_grp['CAN2']['DTS_Status1'],use_display_names=False)
df_DTS_Status1.head()

,DTS_BrkPedlPosn,DTS_VehicleSpeed,DTS_AccrPedlPosn,DTS_Status1_checksum,DTS_Status1_counter,DT_ReadyToSleep,DTS_SystemStatus,DTS_AccrPedlSw,DTS_IgnitionStatus,DTS_BrkPedlSw,DTS_VehMoveDir,DTS_ReadyToSleep,DTS_ReadyToDrive
timestamps,,,,,,,,,,,,,
0.000000,100.0,41.21,0.0,137,9,b'NOT_READY',b'On',b'Pedal Released',b'ON',b'Pedal Depressed',b'FORWARD',b'NOT_READY',b'READY'
0.050137,100.0,40.77,0.0,208,10,b'NOT_READY',b'On',b'Pedal Released',b'ON',b'Pedal Depressed',b'FORWARD',b'NOT_READY',b'READY'
0.100196,100.0,40.33,0.0,27,10,b'NOT_READY',b'On',b'Pedal Released',b'ON',b'Pedal Depressed',b'FORWARD',b'NOT_READY',b'READY'
0.150192,100.0,39.91,0.0,101,10,b'NOT_READY',b'On',b'Pedal Released',b'ON',b'Pedal Depressed',b'FORWARD',b'NOT_READY',b'READY'
0.200211,100.0,39.45,0.0,9,10,b'NOT_READY',b'On',b'Pedal Released',b'ON',b'Pedal Depressed',b'FORWARD',b'NOT_READY',b'READY'


In [25]:
msgs1_df;

defaultdict(<function __main__.<lambda>()>,
            {'CAN1': defaultdict(list,
                         {'DTS_Status2':             DTS_Status2_counter  DTS_Status2_checksum  DTS_IDU_HV_CheckState  \
                          timestamps                                                                     
                          0.000000                      0                   108                      1   
                          0.099946                      0                   113                      1   
                          0.200000                      0                    86                      1   
                          0.299945                      0                   240                      1   
                          0.399970                      1                   237                      1   
                          ...                         ...                   ...                    ...   
                          60.603976                   